# ST IT Cloud - Data and Analytics Test LV.3

Esse teste deve avaliar a qualidade técnica na manipulacão de dados.

## Passo a passo

- Disponibilizamos aqui 1 case para ser solucionado, leia os enunciados dos problemas, desenvolva os programas, utilizando a **stack definida durante o processo seletivo**, para entregar os dados de acordo com os requisitos descritos abaixo.


**Faz parte dos critérios de avaliacão a pontualidade da entrega. Implemente até onde for possível dentro do prazo acordado.**

**Os dados de pessoas foram gerados de forma aleatória, utilizando a biblioteca FakerJS, FakerJS-BR e Faker**

LEMBRE-SE: A entrega deve conter TODOS os passos para o avaliador executar o programa (keep it simple).


# TESTE PRÁTICO

**Problema**: Você está recebendo o arquivo 'dados_cadastrais_fake.csv' que contem dados cadastrais de clientes, mas para que análises ou relatórios sejam feitos é necessário limpar e normalizar os dados. Além disso, existe uma coluna com o número de cpf e outra com cnpj, você precisará padronizar deixando apenas dígitos em formato string (sem caracteres especiais), implementar uma forma de verificar se tais documentos são válidos sendo que a informação deve se adicionada ao dataframe em outras duas novas colunas.

Após a normalização, gere reports que respondam as seguintes perguntas:
- Quantos clientes temos nessa base?
- Qual a média de idade dos clientes?
- Quantos clientes nessa base pertencem a cada estado?
- Quantos CPFs válidos e inválidos foram encontrados?
- Quantos CNPJs válidos e inválidos foram encontrados?

Ao final gere um arquivo no formato csv e um outro arquivo no formato parquet chamado (problema1_normalizado), eles serão destinados para pessoas distintas.

In [3]:
## Importando as bibliotecas para serem utilizadas no teste prático
import pandas as pd
import pandasql as ps
from functions import validador_cpf, validador_cnpj

In [5]:
## Criando DF através do CSV fornecido

DadosCadastraisDF = pd.read_csv('dados_cadastrais_fake.csv', sep=';')

,nomes,idade,cidade,estado,cpf,cnpj
0,Dennis Daniels,31,ACRELÂNDIA,AC,97566536800,06589184909526
1,Leah Becker,42,ÁGUA BRANCA,AL,425.263.807-07,25.673.336/2350-20
2,Sally Ford,18,ALVARÃES,AM,34647754103,26543101702989
3,Colleen Duncan,21,SERRA DO NAVIO,AP,252.531.560-03,19.062.080/5100-98
4,Jeff Stephenson,73,ABAÍRA,BA,49668886542,97794530015384
...,...,...,...,...,...,...
9995,Rebekah Mitchell PhD,55,ABAIARA,CE,744.822.622-34,16.740.076/9329-75
9996,Lisa Parrish Jr.,73,Brasília,DF,10683395190,32246978843482
9997,Michael Young MD,87,AFONSO CLÁUDIO,ES,538.223.638-04,86.601.303/7580-88
9998,Kevin Watson DDS,82,ABADIA DE GOIÁS,GO,11632512408,08651414023648


In [6]:
## Normalização dos dados coluna 'nomes'

DadosCadastraisDF['nomes'] = DadosCadastraisDF['nomes'].str.replace('Dr.', '', regex=True)
DadosCadastraisDF['nomes'] = DadosCadastraisDF['nomes'].str.replace('Mr.', '', regex=True)
DadosCadastraisDF['nomes'] = DadosCadastraisDF['nomes'].str.replace('Mrs.', '', regex=True)
DadosCadastraisDF['nomes'] = DadosCadastraisDF['nomes'].str.replace('Ms.', '', regex=True)
DadosCadastraisDF['nomes'] = DadosCadastraisDF['nomes'].str.replace('Miss', '', regex=True)
DadosCadastraisDF['nomes'] = DadosCadastraisDF['nomes'].str.replace('MD', '', regex=True)
DadosCadastraisDF['nomes'] = DadosCadastraisDF['nomes'].str.replace('PhD', '', regex=True)
DadosCadastraisDF['nomes'] = DadosCadastraisDF['nomes'].str.replace('DDS', '', regex=True)
DadosCadastraisDF['nomes'] = DadosCadastraisDF['nomes'].str.replace('DVM', '', regex=True)
DadosCadastraisDF['nomes'] = DadosCadastraisDF['nomes'].str.replace('.', '', regex=True)



In [7]:
## Normalização dos dados coluna 'estado'

DadosCadastraisDF['estado'] = DadosCadastraisDF['estado'].str.replace('distrito federal', 'DF', regex=True)
DadosCadastraisDF['estado'] = DadosCadastraisDF['estado'].str.replace('sao  paulo', 'SP', regex=True)
DadosCadastraisDF['estado'] = DadosCadastraisDF['estado'].str.replace('rio de  janeiro ', 'RJ', regex=True)
DadosCadastraisDF['estado'] = DadosCadastraisDF['estado'].str.replace('MINAS GERAI', 'MG', regex=True)
DadosCadastraisDF['estado'] = DadosCadastraisDF['estado'].str.replace('MINAS GERAIs', 'MG', regex=True)
DadosCadastraisDF['estado'] = DadosCadastraisDF['estado'].str.replace('MGs', 'MG', regex=True)
DadosCadastraisDF['estado'] = DadosCadastraisDF['estado'].str.replace('são  paulo', 'SP', regex=True)

In [8]:
## Removendo os caracteres especiais das colunas cpf e cnpj

DadosCadastraisDF['cpf'] = DadosCadastraisDF['cpf'].str.replace('.', '', regex=True)
DadosCadastraisDF['cpf'] = DadosCadastraisDF['cpf'].str.replace('-', '', regex=True)
DadosCadastraisDF['cnpj'] = DadosCadastraisDF['cnpj'].str.replace('/', '', regex=True)
DadosCadastraisDF['cnpj'] = DadosCadastraisDF['cnpj'].str.replace('.', '', regex=True)
DadosCadastraisDF['cnpj'] = DadosCadastraisDF['cnpj'].str.replace('-', '', regex=True)


In [16]:
## Criando as colunas para validar os dados de cpf e cnpj
DadosCadastraisDF['cpf_valido'] = ''
DadosCadastraisDF['cnpj_valido'] = ''

for i, item in enumerate(DadosCadastraisDF['cpf']):
    if validador_cpf(DadosCadastraisDF['cpf'][i]) == True:
        DadosCadastraisDF['cpf_valido'][i] = True
    else:
        DadosCadastraisDF['cpf_valido'][i] = False

for i, item in enumerate(DadosCadastraisDF['cnpj']):
    if validador_cnpj(DadosCadastraisDF['cnpj'][i]) == True:
        DadosCadastraisDF['cnpj_valido'][i] = True
    else:
        DadosCadastraisDF['cnpj_valido'][i] = False

C:\Users\aland\AppData\Local\Temp\ipykernel_8772\3900002056.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DadosCadastraisDF['cpf_valido'][i] = True
C:\Users\aland\AppData\Local\Temp\ipykernel_8772\3900002056.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DadosCadastraisDF['cpf_valido'][i] = False
C:\Users\aland\AppData\Local\Temp\ipykernel_8772\3900002056.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DadosCadastraisDF

In [25]:
## Quantos clientes temos na nessa base?

Total = DadosCadastraisDF['nomes'].count()
print(f'Temos um total de {Total} clientes, cadastrados nessa base!')

Temos um total de 10000 clientes, cadastrados nessa base!


In [29]:
## Qual a média de idade dos clientes?

Media = DadosCadastraisDF['idade'].mean()
print(f'A idade média dos clientes é {Media:.2f}')

A idade média dos clientes é 53.78


In [38]:
## Qual a quantidade de clientes por estado?

query = """SELECT estado, count(*) as total_clientes FROM DadosCadastraisDF GROUP BY estado"""

print(ps.sqldf(query,locals()))

   estado  total_clientes
0      AC             371
1      AL             371
2      AM             371
3      AP             371
4      BA             371
5      CE             371
6      DF             371
7      ES             371
8      GO             371
9      MA             371
10     MG             370
11     MS             370
12     MT             370
13     PA             370
14     PB             370
15     PE             370
16     PI             370
17     PR             370
18     RJ             370
19     RN             370
20     RO             370
21     RR             370
22     RS             370
23     SC             370
24     SE             370
25     SP             370
26     TO             370


In [40]:
## Qual a quantidade de clientes com cpf válidos?

query1 = """SELECT cpf_valido, count(*) as total_clientes FROM DadosCadastraisDF GROUP BY cpf_valido"""

print(ps.sqldf(query1,locals()))

   cpf_valido  total_clientes
0           0               1
1           1            9999


In [41]:
## Qual a quantidade de clientes com cnpj válidos?

query2 = """SELECT cnpj_valido, count(*) as total_clientes FROM DadosCadastraisDF GROUP BY cnpj_valido"""

print(ps.sqldf(query2,locals()))

   cnpj_valido  total_clientes
0            1           10000


In [47]:
## Salvando o resultado em csv e parquet

DadosCadastraisDF.to_parquet(
    'problema1_normalizado.parquet',
    engine='pyarrow'
    )

DadosCadastraisDF.to_csv(
    'problema1_normalizado.csv',
    sep=';'
)